<h1> Cleavage Site and TAP Prediction </h1>


This tutorial illustrates the use of Fred2 to predict the steps of the HLA-I antigen processing pathway including proteasomal cleavage and TAP transport. Fred2 offers a long list of prediction methods and was  designed in such a way that extending Fred2 with your favorite method is easy.

This tutorial will entail:
- Simple cleavage site/fragment prediction from a list of peptide sequences and protein sequences
- Simple TAP prediction methods
- Consensus prediction of proteasomal cleavage, TAP, and HLA binding to model the complete antigen processing pathway


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from Fred2.Core import Protein, Peptide, Allele
from Fred2.Core import generate_peptides_from_protein
from Fred2.IO import read_fasta
from Fred2.CleavagePrediction import CleavageSitePredictorFactory, CleavageFragmentPredictorFactory
from Fred2.TAPPrediction import TAPPredictorFactory

<h2> Chapter 1: Cleavage Prediction </h2>
<br/>
Fred2 offers a comprehensive list of state-of-the art proteasomal cleavage prediction methods. Usually one distinguishes cleavage methods into cleavage site and cleavage fragment prediction. Cleavage site prediction methods predict each possible cleavage site within a given amino acid sequence, whereas cleavage fragment prediction methods predict the likelihood of a peptide fragment of being a product of proteasomal cleavage. Additionally, the source of the training data are often also quite different. The majority of prediction tools was train on in vitro data of up to three fully analyzed proteins and distinguish between constitutive and immunoproteasomal cleavage. Some prediction methods use natural HLA ligands as training data as they are products of antigen processing pathway and therefore also of cleavage events.
<br/><br/>
But all methods start with reading in protein sequences. Fred2 offers several ways of defining Proteins. We can either directly initialize a `Fred2.Core.Protein` object by specifying a amino acid sequence and optionally a progenitor gene and transcript id, as well as the progenitor `Fred2.Core.Transcript` object, or we can directly read in proteins from FASTA files by using `Fred2.IO.read_fasta`.  

In [ ]:
protein = Protein("AAAAAAAAAAA",_gene_id="Dummy", _transcript_id="Dummy")
proteins = read_fasta("./data/proteins.fasta", id_position=3, type=Protein)
protein

Once we have a protein sequence to work with, we can specify the cleavage site prediction method of our choice. Fred2 offers one entry point for each type of prediction methods via so called factories. For cleavage site prediction it is `CleavageSitePredictorFactory`. To get an overview which prediction methods are currently implemented, we can use `CleavageSitePredictorFactory` as follows:

In [ ]:
for name,version in CleavageSitePredictorFactory.available_methods().iteritems():
    print name, ",".join(version)

Lets select `PCM` for example and make predictions:

In [ ]:
pcm = CleavageSitePredictorFactory("PCM")
site_result = pcm.predict(proteins)
site_result.head()

To specify a particular version of a prediction method, we can use the flag `version=""` when calling the PredictorFactories. If we do not specify any version, Fred2 will initialize the most recent version that is supported.

In [ ]:
pcm = CleavageSitePredictorFactory("PCM", version="1.0")
site_result = pcm.predict(proteins)
site_result.head()

External tools like `NetChop` offer two additional flags when calling `.predict()`, `command="/path/to/binary"` and `options="command options"`. `command=""` specifies the path to an alternative binary that should be used instead of the one which is globally registered. With options you can specify additional commands that will directly be passed to the command line call without any sanity checks.

In [ ]:
pcm = CleavageSitePredictorFactory("NetChop")
site_result = pcm.predict(proteins, options="")
site_result.head()

For CleavageFragment prediction we first have to generate peptides from protein sequences with `Fred2.Core.generate_peptides_from_proteins`. Fred2 currently supports only one CleavageFragment prediction methods proposed by Ginodi et al. <a href="http://bioinformatics.oxfordjournals.org/content/24/4/477.long">(Ginodi, et al.(2008) Bioinformatics 24(4))</a>, which supports only 11mers (9mer epitopes and two flaking amino acids).

In [ ]:
pep = generate_peptides_from_protein(proteins, 11)
CleavageFragmentPredictorFactory("Ginodi").predict(pep).head()

The result object is based on pandas' `DataFrame`, thus all possibilities of manipulating the results pandas offers are possible, including rudimentary plotting capabilities.

In [ ]:
import matplotlib.pyplot as plt

f, a = plt.subplots(len(site_result.index.levels[0]),1)

for i,r in enumerate(site_result.index.levels[0]):
    site_result.xs(r).plot(kind='bar', ax=a[i]).set_xticklabels(site_result.loc[(r,slice(None)),"Seq"], rotation=0)


We also can combine several prediction results of the same type via `CleavageSitePredictionResults.merge_results` (Note this function returns a merge Result DataFrame):

In [ ]:
import pandas as pd
import numpy

site_result2 = CleavageSitePredictorFactory("proteasmm_c").predict(proteins)
merged_result = site_result.merge_results([site_result2])
merged_result.head(7)

We can also filter the results based on multiple expressions with `CleavageSitePredictionResults.filter_result`.

In [ ]:
comp = lambda x,y: x > y
expressions=[("pcm",comp,0)]

merged_result.filter_result(expressions)

<h2> Chapter 2: TAP prediction </h2>

Fred2 offers only limited prediction methods for TAP prediction, due to lack of publicly available methods.

In [ ]:
for name,version in TAPPredictorFactory.available_methods().iteritems():
    print name, ",".join(version)

For TAP prediction, we first have to generate peptides. Lets take the proteins we already imported and generate 9mers, as these two methods only support 9mer peptides.

In [ ]:
pep = generate_peptides_from_protein(proteins,9)
tap_result = TAPPredictorFactory("SVMTAP").predict(pep)
tap_result.head()

Again we can do all rudimentary operations on the result object as with the cleavage result objects, including merging and filtering.

In [ ]:
tap_result2 = TAPPredictorFactory("TAPDoytchinova").predict(pep[:15])
tap_result.merge_results(tap_result2).head()

In [ ]:
from operator import ge

tap_result.filter_result([("svmtap",ge, -30)])

<h2> Chapter 3: Consensus prediction for natural ligand prediction </h2>
<br/>
Proteasomal cleavage, TAP prediction, as well as HLA binding prediction have been combined to increase the specificity of predicting natural processed HLA ligands. One example is `WAPP` <a href="http://www.ncbi.nlm.nih.gov/pmc/articles/PMC2279325/">(Dönnes, et al.(2005). Protein Science 14(8))</a>, which uses proteasomal cleavage and TAP prediction methods to filter for possibly processed peptides.

The same approach can be implemented with Fred2. We start again with the two protein sequences and exemplify the workflows for CleavageFragmentPrediction and CleavageSitePrediction methods.


<h4> Antigen processing prediction with CleavageFragment prediction </h4>
<br/>
We will use `PSSMGinodi`, `SVMTAP`, and `UniTope` for prediction for HLA-A*02:01. For `PSSMGionid` and `SVMTAP` we use a threshold of -15 and -30 respectively.


In [ ]:
from operator import ge
from Fred2.Core import Allele
from Fred2.EpitopePrediction import EpitopePredictorFactory

allele = Allele("HLA-A*02:01")

pep = generate_peptides_from_protein(proteins,11)
print "Number of peptides: ", len(pep)

#cleavage prediction and filtering
df_cl = CleavageFragmentPredictorFactory("Ginodi").predict(pep).filter_result(("ginodi",ge,-15))
print "Number of peptides after proteasomal cleavage: ", len(df_cl)

#tap prediction and filtering
df_tap = TAPPredictorFactory("SVMTAP").predict(df_cl.index).filter_result(("svmtap",ge,-30))
print "Number of peptides after TAP transport: ", len(df_tap)

#epitope prediction and filtering
df_epi = EpitopePredictorFactory("UniTope").predict(df_tap.index,alleles=allele)
df_epi

Based on this analysis, there are no natural ligands predicted for the two test proteins.

<h4> Antigen processing prediction with CleavageSite prediction </h4>
<br/>
We will use `PCM`, `SVMTAP`, and `SVMHC` for prediction for HLA-A*02:01 like in the original work of WAPP. For `PCM` and `SVMTAP` we use a threshold of −4.8 and −27 respectively.


In [ ]:
from operator import ge
from Fred2.Core import Allele,Protein
from Fred2.EpitopePrediction import EpitopePredictorFactory

allele = Allele("HLA-A*02:01")



#cleavage prediction and filtering
df_cl = CleavageSitePredictorFactory("PCM").predict(proteins).filter_result(("pcm",ge,-4.8))

print "Number of peptides after proteasomal cleavage: ", len(df_cl)

#since we only predicted possible cleavage site, we now have to generate all possible peptides
#peptides
pep_dic = {}
for p in proteins:
    for i in df_cl.loc[(p.transcript_id,slice(None)),:].index.labels[1]:
        if i-8>=0:
            seq = str(p[i-8:i+1])
            pep_dic.setdefault(seq, []).append(p)
peps = [Peptide(seq,proteins={pp.transcript_id:pp for pp in p}) for seq, p in pep_dic.iteritems()]


#tap prediction and filtering
df_tap = TAPPredictorFactory("SVMTAP").predict(peps).filter_result(("svmtap",ge,-27))
print "Number of peptides after TAP transport: ", len(df_tap)

#epitope prediction and filtering
df_epi = EpitopePredictorFactory("SVMHC").predict(df_tap.index,alleles=allele).filter_result(("svmhc",ge,-1.0))
df_epi

Here as well, we did not predict any natural ligands.